In [ ]:
import sys
sys.path.append('..')

In [ ]:
import utils
import utils_preprocess
import utils_spatial_interpolation
import utils_spatial_analysis

import numpy as np
import pandas as pd
import netCDF4 as nc
import matplotlib.pyplot as plt

In [ ]:
# Load the shapefile
path_shape = '/home/saul/workspace/Well_Imputation/groundwater_imputation_api/src/imputation_api/artifacts/aquifer_shapes/Beryl_Enterprise.shp'
aquifer_shape = utils.load_shapefile(path=path_shape)

# STEP 1: Preprocessing PDSI and GLDAS data into Tabular Format

In [ ]:
# Process the pdsi netcdf files to obtain tabular data pickle file
pdsi_source_directory = r'C:\Users\saulg\Desktop\Remote_Data\pdsi'
pdsi_target_directory = r'C:\Users\saulg\Desktop\Remote_Data\pdsi_tabular'

utils_preprocess.process_pdsi_data(
    source_directory=pdsi_source_directory, 
    target_directory=pdsi_target_directory,
    date_start='01/01/1850',
    date_end='12/31/2020',
    )

In [ ]:
# Process the gldas netcdf files to obtain tabular data pickle file
gldas_source_directory = r'C:\Users\saulg\Desktop\Remote_Data\GLDAS'
gldas_target_directory = r'C:\Users\saulg\Desktop\Remote_Data\gldas_tabular'

utils_preprocess.process_gldas_data(
    source_directory=gldas_source_directory, 
    target_directory=gldas_target_directory,
    )

# Step 2: Transform PDSI, GLDAS, and Well Observations into format for ML

In [ ]:
# Parse pdsi data and save it
directory_pdsi = r"/mnt/c/Users/saulg/Desktop/Remote_Data/pdsi_tabular"
pdsi:dict = utils.pull_relevant_data(
    shape=aquifer_shape, 
    dataset_name="PDSI", 
    dataset_directory=directory_pdsi
    )
utils.save_pickle(
    data=pdsi, 
    file_name="pdsi_data.pickle", 
    directory="/home/saul/workspace/Well_Imputation/groundwater_imputation_api/src/imputation_api/artifacts/dataset_outputs",
    protocol=3)

In [ ]:
# Parse the GLDAS data and save it
directory_gldas = r"/mnt/c/Users/saulg/Desktop/Remote_Data/gldas_tabular"
gldas:dict = utils.pull_relevant_data(
    shape=aquifer_shape, 
    dataset_name="GLDAS", 
    dataset_directory=directory_gldas
    )
utils.save_pickle(
    data=gldas, 
    file_name="gldas_data.pickle", 
    directory="/home/saul/workspace/Well_Imputation/groundwater_imputation_api/src/imputation_api/artifacts/dataset_outputs",
    protocol=3)

In [ ]:
# Process well data from csv files
well_locations = pd.read_csv("/home/saul/workspace/Well_Imputation/groundwater_imputation_api/src/imputation_api/artifacts/aquifer_data/EscalanteBerylLocation.csv")
well_timeseries = pd.read_csv("/home/saul/workspace/Well_Imputation/groundwater_imputation_api/src/imputation_api/artifacts/aquifer_data/EscalanteBerylTimeseries.csv")
data:dict = utils.transform_well_data(
    well_timeseries=well_timeseries, 
    well_locations=well_locations,
    timeseries_name="timeseries",
    locations_name="locations",
    )
utils.save_pickle(
    data=data, 
    file_name="beryl_enterprise_data.pickle", 
    directory="/home/saul/workspace/Well_Imputation/groundwater_imputation_api/src/imputation_api/artifacts/dataset_outputs", 
    protocol=3,
    )


In [ ]:
# Plot the timeseries data to see if it looks reasonable
plt.plot(data["timeseries"], '-.')
plt.show()

# Step 3: Develop initial imputation model

# Step 4: Develop iterative refinement model

# Step 5: Analyze spatial characteristics of imputation model

In [ ]:
data_path = "/home/saul/workspace/Well_Imputation/groundwater_imputation_api/src/imputation_api/artifacts/dataset_outputs/beryl_enterprise_well_data_imputation_iteration_1.pickle"

utils_spatial_interpolation.kriging_interpolation(
    data_pickle_path = data_path,
    shape_file_path = path_shape,
    n_x_cells=100,
    influence_distance=0.125,
    monthly_time_step=1,
    netcdf_filename="beryl_enterprise_spatial_analysis_iteration_1.nc",
    directory="/home/saul/workspace/Well_Imputation/groundwater_imputation_api/src/imputation_api/artifacts/dataset_outputs"
    )

# Step 6: Calculate Storage Change

In [ ]:
raster = nc.Dataset("/home/saul/workspace/Well_Imputation/groundwater_imputation_api/src/imputation_api/artifacts/dataset_outputs/beryl_enterprise_spatial_analysis_iteration_1.nc", 'r')

spatial_analysis = utils_spatial_analysis.StorageChangeCalculator(
    units="English",
    storage_coefficient=0.2,
    anisotropic="x",
)
storage_change = spatial_analysis.calulate_storage_curve(
    raster=raster, 
    date_range_filter=("1948-01-01", "1978-01-01"), # if you need to filter dates outside of original time range
    )

plt.plot(storage_change, '-.')